## Hybrid Neural-Net GARCH Models

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf

start_date = '2018-01-01'
data = yf.download(
    '^GSPC',
    start = start_date,
    interval = '1d',
    progress = False
)

### 1. NNGARCH

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

class NNGARCH(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NNGARCH, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, 1)  
        self.sigmoid = nn.Sigmoid() 

    def forward(self, x):
        out = self.fc1(x)
        out = self.sigmoid(out) 
        out = self.fc2(out)
        return out

### 2. Recurrent Neural Net & GARCH

#### 2-1. RNN-GARCH

#### 2-2. $\alpha$RNN-GARCH

#### 2-3. $\alpha_t$RNN-GARCH

### 3. Market Long Memories

#### 3-1. FIRNN-GARCH

#### 3-2. $\alpha$FIRNN-GARCH

#### 3-3. $\alpha_t$FIRNN-GARCH